In [13]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [14]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [15]:
COLLECTION_NAME = 'data_true_1'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[863867036,
 5695632,
 11773172,
 807257672,
 31072167,
 127923406,
 37056992,
 132345709,
 1220937768077012992,
 219547391,
 310427784,
 18238495,
 2608043178,
 106753810,
 3300894361,
 859191723917955072,
 46286512,
 4721921953,
 766476350,
 487188938,
 378104863,
 1699793220,
 785332570233765888,
 359211782,
 3150308066,
 37523601,
 243046416,
 1338431608697720833,
 1363124777146343427,
 1182499777688166402,
 1224217943547895808,
 1164263100616495104,
 2272162736,
 974419362,
 2428776326,
 2778707184,
 1010127462,
 830035634,
 86755842,
 394022499,
 216878342,
 163381359,
 4584282501,
 864566485188005888,
 110199502,
 2816104456,
 65782197,
 1684815182,
 3222493945,
 259441712,
 492884622,
 163578074,
 15254192,
 1110347290118684674,
 717034192,
 3419141,
 145518909,
 1242138082742169607,
 3343040429,
 1303406571968507915,
 374130998,
 21523213,
 897929271179583488,
 734807417867755521,
 168720269,
 1302444748511367169,
 521894235,
 832170559,
 470291891,
 324643436,
 88792639923077

In [16]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{5695632}
5695632 Fri May 14 20:19:46 +0000 2021
5695632
5695632
Sun May 16 03:59:02 +0000 2021
113956.0
Sat May 15 19:30:13 +0000 2021
83427.0
Sat May 15 00:45:46 +0000 2021
15960.0
Sun May 16 01:19:45 +0000 2021
104399.0
Sat May 15 03:22:02 +0000 2021
25336.0
Sat May 15 01:12:55 +0000 2021
17589.0
Sat May 15 22:54:56 +0000 2021
95710.0
Sun May 16 00:04:46 +0000 2021
99900.0
Sat May 15 00:52:05 +0000 2021
16339.0
Sat May 15 13:21:57 +0000 2021
61331.0
Sat May 15 17:59:32 +0000 2021
77986.0
Sat May 15 05:47:45 +0000 2021
34079.0
Sat May 15 11:54:41 +0000 2021
56095.0
Sat May 15 12:33:08 +0000 2021
58402.0
Fri May 14 23:57:59 +0000 2021
13093.0
Sat May 15 14:32:46 +0000 2021
65580.0
 user_id: 5695632, created_at: Fri May 14 20:19:46 +0000 2021, difference_seconds: 0
├──  user_id: 573167714, created_at: Sun May 16 03:59:02 +0000 2021, difference_seconds: 113956.0
├──  user_id: 1295087750258003968, created_at: Sat May 15 19:30:13 +0000 2021, difference_seconds: 83427.0
├──  user_id: 13034

In [17]:
users= [5695632]

In [18]:
for user in users:
    print(user, get_followers_len(user, db))

5695632 386907


In [20]:
for user in users:
    print(user, get_cascade_size(user, 'data_true_1', db))

5695632 511


In [21]:
# Check if the text is same
db['data_true_1'].distinct('retweet_count')

[0, 617]

In [22]:
db['data_true_1'].distinct('retweeted_status.retweet_count')

[617]

In [23]:
db['data_true_1'].distinct('retweeted_status.quoted_status.retweet_count')

[]